# Variational Autoencoder for anomaly detection


### Import

In [13]:
import tensorflow as tf 

import matplotlib.pyplot as plt 
import numpy as np 
 
import os 
import random 
from IPython import display 
import cv2

### Setting parameters 


In [14]:
# parameters for building the model and training
BATCH_SIZE = 1
LATENT_DIM = 128
IMAGE_SIZE = 64

### Load the dataset 

In [11]:
def get_dataset_slices_paths(image_dir):
    ''' returns a list of paths to the image files'''
    image_file_list = os.listdir(image_dir)
    image_paths = [os.path.join(image_dir, fname) for fname in image_file_list]

    return image_paths 

def map_image(image_filename): 
    ''' preprocess the images'''
    img_raw = tf.io.read_file(image_filename)
    image = tf.image.decode_jpeg(img_raw) # depends on the type of images

    image = tf.cast(image, dtype=tf.float32)
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = image/255.0
    image = tf.reshape(image, shape=(IMAGE_SIZE, IMAGE_SIZE,3,))

    return image 

In [8]:
def load_images_from_folder(folder):
    images=[]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None: 
            img = cv2.resize(img, (64,64))
            images.append(img)
    return np.array(images)

folder_path = "/OneDrive/Documents/STUDY/AutoEncoder/training_data/"

image_data = load_images_from_folder(folder_path)

image_data = image_data.astype('float32')/255.0

### Display the images

In [9]:
def display_fabric(dataset, size=4):
    ''' Takes a sample from a dataset batch and plots it in a grid. '''
    dataset = dataset.unbatch().take(size)
    n_cols = 2
    n_rows = size//n_cols+1
    plt.figure(figsize=(5, 5))
    i=0
    for image in dataset:
        i+=1 
        disp_image = np.reshape(image, (64,64,3))
        plt.subplot(n_rows, n_cols, i)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(disp_image)

def display_one_row(dis_images, offset, shape=(64,64)):
    ''' Display a row of images. '''
    for i, img in enumerate(dis_images):
        plt.subplot(2,2,offset+i+1)
        plt.xticks([])
        plt.yticks([])
        image = np.reshape(img, shape)
        plt.imshow(img)

def display_results(dis_input_images, dis_predicted):
    ''' Display input and predicted images. '''
    plt.figure(figsize=(5,5))
    display_one_row(dis_input_images, 0, shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    display_one_row(dis_predicted, 20, shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

In [ ]:
import matplotlib.pyplot as plt

def display_images(images, num_images=4, figsize=(10, 5)):
    plt.figure(figsize=figsize)
    for i in range(num_images):
        ax = plt.subplot(1, num_images, i + 1)
        plt.imshow(images[i], cmap='gray')  # Adjust the colormap based on image color format
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Assuming image_data is already loaded and preprocessed
# Display the first 5 images from image_data
display_images(image_data, num_images=5)

In [11]:
display_fabric(image_data, size=2)

AttributeError: 'numpy.ndarray' object has no attribute 'unbatch'

### Build the Model 

#### Sampling Class

This layer provide the Gaussian noise input along with the mean ($\mu $) and standard deviation ($\sigma$) of the encoder's output, according to the following equation:
**$$z = \mu + e^{0.5\sigma} * \epsilon $$**
($\epsilon$ = random sample)

In [15]:
class Sampling(tf.keras.layers.Layer):
    def call(self, inputs):
        ''' Generates a random sample and combines with the encoder output
        Args:
        inputs - output tensor from the encoder 
        Returns: 
        'inputs' tensors combined with a random sample
        '''
        mu, sigma = inputs
        batch = tf.shape(mu)[0]
        dim = tf.shape(mu)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        z = mu + tf.exp(0.5*sigma)*epsilon
        return z 
    

#### Encoder Layers


In [17]:
def encoder_layers(inputs, latent_dim):
    ''' Defines the encoder's layers.
    Args: 
        input: a batch from the dataset
        latent_dim: dimensionality of the latent space
        
    Returns:
        mu: learned mean
        sigma: learned standard deviation
        feature_shape: shape of the features before flattening
    '''

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, 
                               padding='same', name='encode_conv1')(inputs)
    x = tf.keras.layers.LeakyReLU(0.1)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, 
                               padding='same', name='encode_conv2')(x)
    x = tf.keras.layers.LeakyReLU(0.1)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, 
                               padding='same', name='encode_conv3')(x)
    x = tf.keras.layers.LeakyReLU(0.1)(x)
    features = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Flatten(name='encoded_flatten')(features)
    x = tf.keras.layers.Dense(1024, activation='relu', name='encode_dense')(x)
    x = tf.keras.layers.BatchNormalization()(x)

    mu = tf.keras.layers.Dense(latent_dim, name='latent_mu')(x)
    sigma = tf.keras.layers.Dense(latent_dim, name='latent_sigma')(x)
    return mu, sigma, features.shape



#### Encoder Model

In [18]:
def encoder_model(latent_dim, input_shape):
    '''Define the encoder model with the Sampling layer
    Args:
        latent_dim: dimensionality of the latent space
        input_shape: shape of a batch from the dataset
    Returns:
        model: the encoder model
        feature_shape: shape of the features before flattening
    '''
    inputs = tf.keras.layers.Input(shape=input_shape)
    mu, sigma, feature_shape = encoder_layers(inputs=inputs, latent_dim=latent_dim)
    z = Sampling()((mu, sigma))
    # Trong class Sampling ko co __init__ >> Sampling()
    # Func call co param "inputs", inputs = mu, sigma >> Sampling()((mu, sigma))
    # hay inputs = tuple cua mu & sigma
    model = tf.keras.Model(inputs=inputs, outputs=[mu, sigma, z])
    # [mu, sigma, z] are the latent representations
    model.summary()
    return model, feature_shape

#### Decoder Layers

In [19]:
def decoder_layers(inputs, feature_shape):
    ''' Define the decoder layers
    Args: 
        inputs: output of the encoder
        feature_shape: shape of the features before flattening

    Returns:
        Tensor containing the decoded output
        '''
    units = feature_shape[1] * feature_shape[2] * feature_shape[3]
    x = tf.keras.layers.Dense(units, activation='relu', name='decode_dense1')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Reshape((feature_shape[1], feature_shape[2], feature_shape[3]), name='decode_reshape')(x)
    
    # Upsample the features back to the original dimension
    x = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='same', activation='relu', name='decode_conv_transpose1')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='decode_conv_transpose2')(x)
    x = tf.keras.layers.BatchNormalization()(x)    
    x = tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='decode_conv_transpose3')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2DTranspose(filters=3, kernel_size=3, strides=1, padding='same', activation='sigmoid')(x)

    return x

#### Decoder Model

In [20]:
def decoder_model(latent_dim, feature_shape):
    '''Defines the decoder model.
    Args: 
        latent_dim: dimensionality of the latent space
        conv_shape: shape of the features before flattening
    Returns:
        model: the decoder model 
    '''
    inputs = tf.keras.layers.Input(shape=(latent_dim,))
    outputs = decoder_layers(inputs, feature_shape)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.summary()
    return model

#### VAE 

In [21]:
def vae_model(encoder, decoder, input_shape):
    '''
    Defines the VAE model:
    Args:
        encoder: encoder_model
        decoder: decoder_model
        input_shape: shape of the dataset_batch
    Return:
        VAE model
    '''
    inputs = tf.keras.layers.Input(shape=input_shape)

    mu, sigma, z = encoder(inputs)
    reconstructed = decoder(z)

    model = tf.keras.Model(inputs=inputs, outputs=reconstructed)
    return model 

In [22]:
def get_models(input_shape, latent_dim):
    encoder, feature_shape = encoder_model(latent_dim, input_shape)
    decoder = decoder_model(latent_dim, feature_shape)
    vae = vae_model(encoder, decoder, input_shape)

    return encoder, decoder, vae 


In [23]:
encoder, decoder, vae = get_models(input_shape=(64,64,3,), latent_dim=LATENT_DIM)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 encode_conv1 (Conv2D)          (None, 32, 32, 32)   896         ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 32, 32, 32)   0           ['encode_conv1[0][0]']           
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 32)  128         ['leaky_re_lu[0][0]']            
 alization)                                                                                   

#### Train the model

#### Training

In [24]:
train_steps = len(image_data)//BATCH_SIZE
val_steps = len(image_data)//BATCH_SIZE
vae.compile(optimizer='adam', metrics=['accuracy'], loss="mse")

In [29]:
vae.fit(image_data, image_data, epochs=5000, batch_size=2, shuffle=True)

Epoch 1/5000
5/5 [==============================] - 1s 93ms/step - loss: 0.0272 - accuracy: 0.5566
Epoch 2/5000
5/5 [==============================] - 1s 103ms/step - loss: 0.0194 - accuracy: 0.5651
Epoch 3/5000
5/5 [==============================] - 0s 94ms/step - loss: 0.0202 - accuracy: 0.5683
Epoch 4/5000
5/5 [==============================] - 1s 106ms/step - loss: 0.0207 - accuracy: 0.5651
Epoch 5/5000
5/5 [==============================] - 0s 63ms/step - loss: 0.0221 - accuracy: 0.5675
Epoch 6/5000
5/5 [==============================] - 0s 66ms/step - loss: 0.0223 - accuracy: 0.5624
Epoch 7/5000
5/5 [==============================] - 0s 65ms/step - loss: 0.0193 - accuracy: 0.5677
Epoch 8/5000
5/5 [==============================] - 0s 62ms/step - loss: 0.0201 - accuracy: 0.5665
Epoch 9/5000
5/5 [==============================] - 0s 61ms/step - loss: 0.0186 - accuracy: 0.5705
Epoch 10/5000
5/5 [==============================] - 0s 68ms/step - loss: 0.0173 - accuracy: 0.5730
Epoch 1

KeyboardInterrupt: 